In [1]:
import requests
import folium
from folium import Circle, PolyLine
import heapq

In [2]:
help_stations_with_phone = [
    {"coordinates": (30.3179409, 78.0500178), "phone_number": "+917617641909"},
    {"coordinates": (30.319395, 78.0503182), "phone_number": "+919389969916"},
    {"coordinates": (30.3250628, 78.0540411), "phone_number": "+917617641909"},
    {"coordinates": (30.3121892, 78.0474883), "phone_number": "+919389969916"},
    {"coordinates": (30.3257766, 78.0428731), "phone_number": "+917617641909"},
    {"coordinates": (30.3301055, 78.0503148), "phone_number": "+919389969916"},
    {"coordinates": (30.3196005, 78.0422984), "phone_number": "+917617641909"},
    {"coordinates": (30.3184253, 78.0424823), "phone_number": "+919389969916"},
    {"coordinates": (30.3207776, 78.0576667), "phone_number": "+917617641909"},
    {"coordinates": (30.3189236, 78.0506355), "phone_number": "+919389969916"},
    {"coordinates": (30.327541, 78.0479398), "phone_number": "+917617641909"},
    {"coordinates": (30.3163536, 78.054412), "phone_number": "+919389969916"},
    {"coordinates": (30.318361, 78.0527356), "phone_number": "+917617641909"}
]

# --- Unload Coordinates ---
help_stations = [station["coordinates"] for station in help_stations_with_phone]
phone = []

In [3]:
user_location = (30.3245, 78.0430)

In [ ]:
def dijkstra(graph, start):
    pq = [(0, start)] 
    distances = {start: 0}
    previous_nodes = {start: None}

    while pq:
        current_distance, current_node = heapq.heappop(pq)

        if current_distance > distances.get(current_node, float('inf')):
            continue

        for neighbor, weight in graph.get(current_node, {}).items():
            distance = current_distance + weight
            if distance < distances.get(neighbor, float('inf')):
                distances[neighbor] = distance
                previous_nodes[neighbor] = current_node
                heapq.heappush(pq, (distance, neighbor))

    return distances, previous_nodes

In [ ]:
def get_osrm_distance(origin, destination):
    url = f"http://router.project-osrm.org/route/v1/foot/{origin[1]},{origin[0]};{destination[1]},{destination[0]}"
    params = {"overview": "full", "geometries": "geojson"}
    response = requests.get(url, params=params)
    data = response.json()
    if 'routes' in data and len(data['routes']) > 0:
        distance_km = data['routes'][0]['distance'] / 1000
        coords = [(coord[1], coord[0]) for coord in data['routes'][0]['geometry']['coordinates']]
        return distance_km, coords
    else:
        return float('inf'), []

In [ ]:
graph = {}
for station in help_stations:
    graph[station] = {}
    for other_station in help_stations:
        if station != other_station: 
            distance, _ = get_osrm_distance(station, other_station)
            graph[station][other_station] = distance

In [ ]:
graph[user_location] = {}
for station in help_stations:
    distance, _ = get_osrm_distance(user_location, station)
    graph[user_location][station] = distance

distances, previous_nodes = dijkstra(graph, user_location)
nearest = sorted(zip(help_stations, distances.values()), key=lambda x: x[1])[:3]

for station, _ in nearest:
    for station_info in help_stations_with_phone:
        if station_info["coordinates"] == station:
            phone.append(station_info["phone_number"])

In [ ]:
print("\n📍 3 Nearest Help Stations (by walking distance):")
for i, (station, dist) in enumerate(nearest):
    print(f"{i+1}. Location: {station}, Distance: {dist:.2f} km, Phone: {phone[i]}")


📍 3 Nearest Help Stations (by walking distance):
1. Location: (30.3179409, 78.0500178), Distance: 0.00 km, Phone: +917617641909
2. Location: (30.3184253, 78.0424823), Distance: 0.59 km, Phone: +919389969916
3. Location: (30.3301055, 78.0503148), Distance: 0.64 km, Phone: +919389969916


In [ ]:
m = folium.Map(location=user_location, zoom_start=15)

folium.Marker(user_location, tooltip="User", icon=folium.Icon(color='blue')).add_to(m)
Circle(user_location, radius=1500, color="blue", fill=False).add_to(m)
for station in help_stations:
    folium.Marker(station, tooltip="Help Station", icon=folium.Icon(color="gray")).add_to(m)

for i, (station, dist) in enumerate(nearest):
    folium.Marker(
        station,
        tooltip=f"Help Station {i+1} ({dist:.2f} km), Phone: {phone[i]}",
        icon=folium.Icon(color="green")
    ).add_to(m)

    _, coords = get_osrm_distance(user_location, station)
    PolyLine(coords, color='green', weight=4, opacity=0.7).add_to(m)

m.save("live_osrm_nearest_help_stations_with_phone.html")
print("✅ Map saved as 'live_osrm_nearest_help_stations_with_phone.html'")
m


✅ Map saved as 'live_osrm_nearest_help_stations_with_phone.html'


In [10]:
print(phone)

['+917617641909', '+919389969916', '+919389969916']


In [ ]:
from twilio.rest import Client

account_sid = 'AC4c024fc964c6236c3b439abd84d49329' 
auth_token = 'cfc033a413a9832a55f70659e5353adb'    

from_phone = '+19137330629' 
message = "Help " 


client = Client(account_sid, auth_token)

for to_phone in phone:
    try:
        message_sent = client.messages.create(
            body=message,
            from_=from_phone,
            to=to_phone
        )
        print(f"Message sent to {to_phone}: {message_sent.sid}")
    except Exception as e:
        print(f"Failed to send message to {to_phone}: {e}")

Message sent to +917617641909: SM6e93c5b6d08d981883eb452c380bad6a
Message sent to +919389969916: SMf55ca23eeb9ecc5755d095b82c37a6cf
Message sent to +919389969916: SM92ddfcdbe59148706694850d0df453a8
